In [15]:
import os
import json
import urllib.request
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service


In [16]:
driver_path = r"F:\Fork_git\Labelling_Menu_Data\menu_scraper\webdriver\chromedriver\chromedriver.exe"


In [17]:
import json
 
# Opening JSON file
f = open('england_city_codes.json')
 
# returns JSON object as 
# a dictionary
england_city_code = json.load(f)
 
 
# Closing file
f.close()

In [18]:
excluded_categories = {'Japanese', 'Singaporean', 'Indian', 'Korean', 'American', 'Type not available', 'Thai', 'Vietnamese', 'British', 'Italian'}

In [19]:
# Update function to get links and names as per the provided HTML snippet
def get_links_and_names(driver):
    # Locate the entire restaurant divs
    restaurant_divs = driver.find_elements(By.XPATH, '//div[@class="yJIls z y M0"]')
    
    data = []
    
    # Iterating through each restaurant div
    for restaurant_div in restaurant_divs:
        try:
            # Finding name and link
            name_link_elem = restaurant_div.find_element(By.XPATH, './/a[@href and @target="_blank" and contains(@class, "BMQDV")]')
            name = name_link_elem.text
            link = name_link_elem.get_attribute('href')
            
            # Finding restaurant type
            # Note: Might not be available for all entries, handle accordingly
            elems = restaurant_div.find_elements(By.XPATH, './/span[@class="YECgr"]')

            restaurant_type = "Type not available"
            price = "Price not available"



            # Iterate through the found elements and check for the desired conditions
            for elem in elems:
                text = elem.text
                
                # Check if "Chinese" is in text, then it's a type
                if any(symbol in text.lower() for symbol in ['chinese']):
                    restaurant_type = text
                # Check if any of the currency symbols are in text, then it's a price
                elif any(symbol in text for symbol in ["£", "$", "€"]):  # you can add more currencies here
                    price = text

            categories = {category.strip() for category in restaurant_type.split(',')}        
            if not categories.intersection(excluded_categories):
                # Append the restaurant data to the list
                data.append({
                    'name': name, 
                    'link': link, 
                    'type': restaurant_type, 
                    'price': price
                })
        except Exception as e:
            # Log or print exception info and continue with the next restaurant
            print(f"Skipping a restaurant due to an error: {str(e)}")
            continue
    
    return data


# Update function to get page links as per the provided HTML snippet
def get_page_links(driver):
    # XPath to locate the next page button based on the provided HTML snippet
    next_button = driver.find_elements(By.XPATH, '//a[@data-smoke-attr="pagination-next-arrow"]')
    return [btn.get_attribute('href') for btn in next_button]

In [20]:
# Load the existing JSON file at the start
filename = 'trip_advisor_manchester.json'
if os.path.exists(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        all_shop_data = json.load(f)
else:
    all_shop_data = []

base_page = "https://www.tripadvisor.co.uk/FindRestaurants?geo=187069&cuisines=5379&establishmentTypes=10591&priceTypes=10954%2C10955&broadened=false"

if all_shop_data and all_shop_data[0]['name'] == 'Stopping page':
    # Replace the first element
    base_page = all_shop_data[0]['link']



service = Service(executable_path=driver_path)

options = Options()
# options.add_argument("--headless")  # Enable headless mode
# options.add_argument(f'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(70, 91)}.0.4472.124 Safari/537.36')

driver = webdriver.Chrome(service=service)

In [21]:
driver.get(base_page)

time.sleep(2 + random.random() * 5)  # Random delay between 5 and 10 seconds

print("Please login to the website... and press Enter here to continue.")
input()

Please login to the website... and press Enter here to continue.


''

In [22]:
nav_links = get_page_links(driver)

In [23]:








continue_flag = True

page_count = 1

while continue_flag:
    # Process all the links
    print(f'Accessing page {page_count}')
    time.sleep(5 + random.random() * 5)  # Random delay between 5 and 10 seconds
    nav_links = get_page_links(driver)
    shop_data = get_links_and_names(driver)

    # Update the list with new data, ensuring that no duplicates are added
    for shop in shop_data:
        if shop not in all_shop_data:
            all_shop_data.append(shop)

    page_count += 1

    if len(nav_links) < 1:
        continue_flag = False
    else:
        driver.get(nav_links[0])
        


    






Accessing page 1
Accessing page 2


In [24]:
# stopping_page = nav_links[0]
# # Your new data to insert if condition is met
# new_data = {
#     'name': 'Stopping page', 
#     'link': stopping_page, 
#     'type': '', 
#     'price': ''
# }

# # Check if 'all_shop_data' is not empty and if first element has name 'Stopping page'
# if all_shop_data and all_shop_data[0]['name'] == 'Stopping page':
#     # Replace the first element
#     all_shop_data[0] = new_data
# else:
#     # Otherwise, insert new_data at the beginning of the list
#     all_shop_data.insert(0, new_data)


In [25]:
# Dump the updated list back into the JSON file
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(all_shop_data, f, ensure_ascii=False, indent=4)

print(f"Saved {len(all_shop_data)} total shop data to {filename}.")

Saved 146 total shop data to trip_advisor_manchester.json.


In [26]:
driver.quit()

In [27]:
def random_sleep(min_time=1, max_time=3):
    time.sleep(random.uniform(min_time, max_time))

# Define a function to download and save images from URLs
def download_image(img_url, save_path):
    if not os.path.exists(save_path):
        urllib.request.urlretrieve(img_url, save_path)
        print(f'Saved image: {save_path}')

# Define a function to get image URLs
def get_image_urls(driver):
    elems = driver.find_elements(By.XPATH, '//div[@class="img"]/a/img')
    return [elem.get_attribute('src') for elem in elems]

def get_high_res_image_url(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="main-pic-stage"]/img')))
        elem = driver.find_element(By.XPATH, '//div[@class="main-pic-stage"]/img')
        return elem.get_attribute('src')
    except:
        return None

def go_to_next_image(driver):
    try:
        next_button = driver.find_element(By.XPATH, '//a[@class="next J_pic-next"]')
        ActionChains(driver).click(next_button).perform()
        return True
    except NoSuchElementException:
        return False